## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,Research & Development,23,2,Medical,3,41,4,...,3,3,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,Research & Development,6,1,Medical,4,42,2,...,3,1,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,Research & Development,4,3,Life Sciences,2,87,4,...,4,2,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,Sales,2,3,Medical,4,63,2,...,3,4,0,17,3,2,9,6,0,8


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [3]:
attrition_df["PerformanceRating"].value_counts()

,count
PerformanceRating,
3,1244
4,226


In [4]:
# Fix the classes to create binary output, necessary for our neural network model
# Change for PerformanceRating the 3 to 0, 4 to 1
attrition_df.loc[attrition_df["PerformanceRating"]==3,"PerformanceRating"] = 0
attrition_df.loc[attrition_df["PerformanceRating"]==4,"PerformanceRating"] = 1
attrition_df["PerformanceRating"].unique()

array([0, 1])

In [5]:
attrition_df["Department"].unique()

array(['Sales', 'Research & Development', 'Human Resources'], dtype=object)

In [6]:
attrition_df["EducationField"].unique()

array(['Life Sciences', 'Other', 'Medical', 'Marketing',
       'Technical Degree', 'Human Resources'], dtype=object)

In [7]:
attrition_df["BusinessTravel"].unique()

array(['Travel_Rarely', 'Travel_Frequently', 'Non-Travel'], dtype=object)

In [8]:
# Fix the classes to create binary output, necessary for our neural network model
# Change for Travel_Frequently to 2,  to 0, 4 to 1
attrition_df.loc[attrition_df["BusinessTravel"]=="Travel_Frequently", "BusinessTravel"] = 2
attrition_df.loc[attrition_df["BusinessTravel"]=="Travel_Rarely", "BusinessTravel"] = 1
attrition_df.loc[attrition_df["BusinessTravel"]=="Non-Travel", "BusinessTravel"] = 0

In [9]:
attrition_df["BusinessTravel"].unique()

array([1, 2, 0], dtype=object)

In [10]:
# Convert OverTime to int64
attrition_df['OverTime'] = attrition_df['OverTime'].map({'Yes': 1, 'No': 0})

In [11]:
attrition_df["OverTime"].value_counts()

,count
OverTime,
0,1054
1,416


In [12]:
attrition_df

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,1,Sales,1,2,Life Sciences,2,94,3,...,0,1,0,8,0,1,6,4,0,5
1,49,No,2,Research & Development,8,1,Life Sciences,3,61,2,...,1,4,1,10,3,3,10,7,1,7
2,37,Yes,1,Research & Development,2,2,Other,4,92,2,...,0,2,0,7,3,3,0,0,0,0
3,33,No,2,Research & Development,3,4,Life Sciences,4,56,3,...,0,3,0,8,3,3,8,7,3,0
4,27,No,1,Research & Development,2,1,Medical,1,40,3,...,0,4,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,2,Research & Development,23,2,Medical,3,41,4,...,0,3,1,17,3,3,5,2,0,3
1466,39,No,1,Research & Development,6,1,Medical,4,42,2,...,0,1,1,9,5,3,7,7,1,7
1467,27,No,1,Research & Development,4,3,Life Sciences,2,87,4,...,1,2,1,6,0,3,6,2,0,3
1468,49,No,2,Sales,2,3,Medical,4,63,2,...,0,4,0,17,3,2,9,6,0,8


In [13]:
attrition_df.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'Department', 'DistanceFromHome',
       'Education', 'EducationField', 'EnvironmentSatisfaction', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'NumCompaniesWorked', 'OverTime', 'PercentSalaryHike',
       'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')

In [14]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]

y_df.head(3)

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development


In [15]:
y = y_df[['Attrition', 'Department']]
y

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development
...,...,...
1465,No,Research & Development
1466,No,Research & Development
1467,No,Research & Development
1468,No,Sales


In [16]:
# Create a list of at least 10 column names to use as X data

selected_columns_df = ['Age',
                     'BusinessTravel',
                     'DistanceFromHome',
                     'Education',
                     'EnvironmentSatisfaction',
                     'HourlyRate',
                     'JobInvolvement',
                     'JobLevel', 'JobSatisfaction',
                     'NumCompaniesWorked',
                     'OverTime',
                     'PercentSalaryHike',
                     'PerformanceRating',
                     'RelationshipSatisfaction',
                     'StockOptionLevel',
                     'TotalWorkingYears',
                     'TrainingTimesLastYear',
                     'WorkLifeBalance',
                     'YearsAtCompany',
                     'YearsInCurrentRole',
                     'YearsSinceLastPromotion',
                     'YearsWithCurrManager']


# Create X_df using your selected columns
X_df = attrition_df[selected_columns_df]

# Display the first few rows of X_df
X_df.head()

# Show the data types for X_df
X_df.dtypes


,0
Age,int64
BusinessTravel,object
DistanceFromHome,int64
Education,int64
EnvironmentSatisfaction,int64
HourlyRate,int64
JobInvolvement,int64
JobLevel,int64
JobSatisfaction,int64
NumCompaniesWorked,int64


In [17]:
# X = X_df.copy(
# X.head(3)

In [18]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=1)


In [19]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train['OverTime'].value_counts()

,count
OverTime,
0,798
1,304


In [20]:
# Create a StandardScaler
X_scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [21]:
# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop=None)

# Fit the encoder to the training data
department_encoder.fit(np.array(y_train['Department']).reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data
y_department_train = department_encoder.transform(np.array(y_train['Department']).reshape(-1, 1))
y_department_test = department_encoder.transform(np.array(y_test['Department']).reshape(-1, 1))

y_department_train


array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [22]:
# Create a OneHotEncoder for the Department column
attrition_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop=None)

# Fit the encoder to the training data
attrition_encoder.fit(np.array(y_train['Attrition']).reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data
y_attrition_train = department_encoder.transform(np.array(y_train['Attrition']).reshape(-1, 1))
y_attrition_test = department_encoder.transform(np.array(y_test['Attrition']).reshape(-1, 1))

y_attrition_train

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       ...,
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

## Create, Compile, and Train the Model

In [23]:
# Find the number of columns in the X training data
# number_input_features = len(X_train.columns)

# Create the input layer
#input = layers.Input(number_input_features, name='input')

# Create at least two shared layers
#shared1 = layers.Dense(64, activation='relu', name='shared1')(input)
# shared2 = layers.Dense(128, activation='relu', name='shared2')(shared1)

In [24]:
# Find the number of columns in the X training data
number_input_features = len(X_train.columns)

# Create the input layer
# The shape should be a tuple containing the number of features
input = layers.Input(shape=(number_input_features,), name='input')

# Create at least two shared layers
shared1 = layers.Dense(64, activation='relu', name='shared1')(input)
shared2 = layers.Dense(128, activation='relu', name='shared2')(shared1)

In [25]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(32, activation='relu', name='department_hidden')(shared2)

# Create the output layer
department_output = layers.Dense(y_department_train.shape[1], activation='sigmoid', name='department_output')(department_hidden)


In [26]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(32, activation='relu', name='attrition_hidden')(shared2)

# Create the output layer
attrition_output = layers.Dense(y_attrition_train.shape[1], activation='sigmoid', name='attrition_output')(attrition_hidden)


In [27]:
# Create the model
model = Model(inputs=input, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={
                  'department_output': 'categorical_crossentropy',
                  'attrition_output': 'binary_crossentropy'
              },
              metrics={
                  'department_output': 'accuracy',
                  'attrition_output': 'accuracy'
              })

# Summarize the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)        │ (None, 22)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared1 (Dense)           │ (None, 64)             │          1,472 │ input[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared2 (Dense)           │ (None, 128)            │          8,320 │ shared1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_hidden (Dense) │ (None, 32)             │          4,128 │ shared2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_hidden (Dense)  │ (None, 32)             │          4,128 │ shared2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │             99 │ department_hidden[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 3)              │             99 │ attrition_hidden[0][0] │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 18,246 (71.27 KB)

 Trainable params: 18,246 (71.27 KB)

 Non-trainable params: 0 (0.00 B)

In [28]:
# Train the model
fit_model = model.fit(X_train_scaled, {'department_output': y_department_train, 'attrition_output': y_attrition_train}, epochs=100)

Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - attrition_output_accuracy: 0.0115 - attrition_output_loss: 0.9829 - department_output_accuracy: 0.5317 - department_output_loss: 0.4979 - loss: 1.4811
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - attrition_output_accuracy: 0.0000e+00 - attrition_output_loss: 0.7864 - department_output_accuracy: 0.6061 - department_output_loss: 0.0244 - loss: 0.8107
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.0000e+00 - attrition_output_loss: 0.7251 - department_output_accuracy: 0.6421 - department_output_loss: 0.0050 - loss: 0.7300
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.0000e+00 - attrition_output_loss: 0.7279 - department_output_accuracy: 0.6288 - department_output_loss: 0.0028 - loss: 0.7308
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.0000e+00 - attrition_output_loss: 0.6923 - department_output_accuracy: 0.6418 - department

In [29]:
# Evaluate the model with the testing data
# # Evaluate the model with the testing data
# option 2 test_results = model.evaluate(X_test_scaled, {'department_output': y_department_test, 'attrition_output': y_attrition_test})
# test_results
test_results = model.evaluate(X_test_scaled, {'department_output': y_department_test, 'attrition_output': y_attrition_test})
test_results

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.0249 - attrition_output_loss: 3.4661 - department_output_accuracy: 0.5522 - department_output_loss: 9.9510e-07 - loss: 3.4843      


[3.2563021183013916,
 9.103754905481765e-07,
 3.138093948364258,
 0.02445652149617672,
 0.5760869383811951]

In [30]:
# Print the accuracy for both department and attrition
print(f"Department predictions accuracy: {test_results[3]}")
print(f"Attrition predictions accuracy: {test_results[4]}")

Department predictions accuracy: 0.02445652149617672
Attrition predictions accuracy: 0.5760869383811951


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1.
2.
3.

1. Results presented are sub-optimal when both Attrition and Department considered.  Running with 50 epochs did not provided even lesser performance results.
2. The sigmoid function was applied to leverage the binary output to determine retention question.  There's more considerations to be applied with features and reconsideration to alternatives for sigmoid for the output layer.
3. Improvements I beleive to include rationalizion where possible and revisiting the travel distrance to tenure to determine any relationship which may contribute to retention likelihood.